[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-diffusion-diffusers-colab/blob/main/pytorch_stable_diffusion_2.ipynb)

In [ ]:
save_to = "here" #@param ["here", "discord", "gdrive"]
if save_to == "gdrive":
  from google.colab import drive
  drive.mount('/content/gdrive')

!pip install -U git+https://github.com/huggingface/diffusers
!pip install -U transformers piexif fold_to_ascii ftfy
!pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.14/xformers-0.0.14.dev0-cp37-cp37m-linux_x86_64.whl

import torch, os, gc, requests, json, piexif
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from PIL import Image
from PIL.PngImagePlugin import PngInfo
from fold_to_ascii import fold
metadata = PngInfo()

def closestNumber(n, m):
    q = int(n / m)
    n1 = m * q
    if (n * m) > 0:
        n2 = m * (q + 1)
    else:
        n2 = m * (q - 1)
    if abs(n - n1) < abs(n - n2):
        return n1
    return n2

folder_max_files = 500 #@param {type: 'integer'}
root_folder = "ai_images" #@param {type: 'string'}
if save_to == "gdrive":
  root_folder = f"/content/gdrive/MyDrive/{root_folder}"

if os.path.exists(f"{root_folder}") == False:
    os.mkdir(f"{root_folder}")
image_folder = max([int(f) for f in os.listdir(f"{root_folder}")], default=0)
if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
    os.mkdir(f"{root_folder}/{image_folder:04}")
name = max([int(f[: f.index(".")]) for f in os.listdir(f"{root_folder}/{image_folder:04}")],default=0,)

model_folder = "stabilityai/stable-diffusion-2" #@param ["stabilityai/stable-diffusion-2"] {allow-input: true}

is_tile = False #@param {type: 'boolean'}
if(is_tile):
  def patch_conv(cls):
    init = cls.__init__
    def __init__(self, *args, **kwargs):
      return init(self, *args, **kwargs, padding_mode='circular')
    cls.__init__ = __init__
  patch_conv(torch.nn.Conv2d)

scheduler = EulerDiscreteScheduler.from_pretrained(model_folder, subfolder="scheduler")
pipe = StableDiffusionPipeline.from_pretrained(model_folder, scheduler=scheduler).to("cuda")
pipe.enable_xformers_memory_efficient_attention()

def generate(discord_token, discord_channel_id, discord_user, by, num_inference_steps, guidance_scale, sampler, width, height, prompt, negative_prompt, suffix, image_folder, name):
    width = closestNumber(width, 8)
    height = closestNumber(height, 8)
    metadata.add_text("Prompt", f"{prompt}")
    metadata.add_text("by", f"{by}")
    gc.collect()
    with torch.autocast("cuda"):
      images = pipe(prompt, negative_prompt=negative_prompt, num_inference_steps=num_inference_steps, height=height, width=width, guidance_scale=guidance_scale).images
    image = images[0]
    real_seed = torch.cuda.initial_seed()
    if(suffix == 'png'):
      image.save(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", pnginfo=metadata)
    else:
      zeroth_ifd = {piexif.ImageIFD.ImageDescription: f"{fold(prompt)}", piexif.ImageIFD.Make: f"{fold(by)}", piexif.ImageIFD.Model: f"{model_folder}"}
      exif_dict = {"0th": zeroth_ifd}
      exif_bytes = piexif.dump(exif_dict)
      image.save(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", "JPEG", quality=70, exif=exif_bytes)
    files = {f"{image_folder:04}_{name:04}.{suffix}": open(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}", "rb").read()}
    if save_to == "discord":
      payload = {"content": f"{prompt}\nNegative prompt: {negative_prompt}\nSteps: {num_inference_steps}, Sampler: {sampler}, CFG scale: {guidance_scale}, Seed: {real_seed}, Size: {width}x{height}, Model folder: {model_folder} - {discord_user}"}
      requests.post(f"https://discord.com/api/v9/channels/{discord_channel_id}/messages", data=payload, headers={"authorization": f"Bot {discord_token}"}, files=files)
      os.remove(f"{root_folder}/{image_folder:04}/{name:04}.{suffix}")

In [ ]:
discord_token = "" #@param {type: 'string'}
discord_channel_id = 0 #@param {type: 'integer'}
prompt = "duck" #@param {type: 'string'}
negative_prompt = "" #@param {type: 'string'}
width  = 768 #@param {type: 'integer'}
height  = 768 #@param {type: 'integer'}
guidance_scale = 7.5 #@param {type: 'number'}
num_inference_steps = 50 #@param {type: 'integer'}
suffix = "jpg" #@param ["jpg", "png"]
by = "camenduru" #@param {type: 'string'}
template = {
    "discord_token": discord_token,
    "discord_channel_id": discord_channel_id,
    "by": by,
    "num_inference_steps": num_inference_steps,
    "guidance_scale": guidance_scale,
    "sampler": "PLMS",
    "width": width,
    "height": height,
    "prompt": prompt,
    "negative_prompt": negative_prompt,
    "suffix": suffix
  }
with open("template.json", "w") as outfile:
    outfile.write(json.dumps(template))

is_loop = False #@param {type:"boolean"}

if is_loop:
  while True:
      if name < folder_max_files:
          with open("template.json", "r") as file:
              prompts = file.readlines()
          for prompt in prompts:
              d = json.loads(prompt)
              name += 1
              generate(d["discord_token"], d["discord_channel_id"], "camenduru", d["by"], d["num_inference_steps"], d["guidance_scale"], d["sampler"], d["width"], d["height"], d["prompt"], d["negative_prompt"], d["suffix"], image_folder, name)
      else:
          image_folder += 1
          if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
              os.mkdir(f"{root_folder}/{image_folder:04}")
          name = 0
else:
  if name < folder_max_files:
      with open("template.json", "r") as file:
          prompts = file.readlines()
      for prompt in prompts:
          d = json.loads(prompt)
          name += 1
          generate(d["discord_token"], d["discord_channel_id"], "camenduru", d["by"], d["num_inference_steps"], d["guidance_scale"], d["sampler"], d["width"], d["height"], d["prompt"], d["negative_prompt"], d["suffix"], image_folder, name)
  else:
      image_folder += 1
      if os.path.exists(f"{root_folder}/{image_folder:04}") == False:
          os.mkdir(f"{root_folder}/{image_folder:04}")
      name = 0